In [1]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [77]:
model = tf.keras.Sequential(
[
    tf.keras.layers.InputLayer(input_shape=(224,224,3)),
    tf.keras.layers.Conv2D(96,activation="relu",strides=4,kernel_size=11),
    tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=2),
    tf.keras.layers.Conv2D(256,kernel_size=5,padding="SAME",activation="relu"),
    tf.keras.layers.ZeroPadding2D(padding=(1,1)),
    tf.keras.layers.Conv2D(384,kernel_size = 3,activation="relu"),
    tf.keras.layers.ZeroPadding2D(padding=(1,1)),
    tf.keras.layers.Conv2D(384,kernel_size = 3,activation="relu"),
    tf.keras.layers.ZeroPadding2D(padding=(1,1)),
    tf.keras.layers.Conv2D(256,kernel_size = 3,activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10,activation="softmax")
]
)

In [78]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 26, 26, 256)       614656    
_________________________________________________________________
zero_padding2d_21 (ZeroPaddi (None, 28, 28, 256)       0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 26, 26, 384)       885120    
_________________________________________________________________
zero_padding2d_22 (ZeroPaddi (None, 28, 28, 384)       0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 26, 26, 384)      

In [10]:
DATA_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(fname="flower_photos",origin=DATA_URL,untar=True)

In [11]:
import pathlib
data_dir = pathlib.Path(data_dir)

In [61]:
ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))

In [62]:
for tmp in ds.take(2):
    print(tmp)

tf.Tensor(b'C:\\Users\\Happiness\\.keras\\datasets\\flower_photos\\daisy\\1392946544_115acbb2d9.jpg', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\Happiness\\.keras\\datasets\\flower_photos\\tulips\\8713397358_0505cc0176_n.jpg', shape=(), dtype=string)


In [63]:
import numpy as np

In [64]:
class_lable =  np.array([ item.name for item in data_dir.glob("*") if item.name != "LICENSE.txt" ])
## 转换lable为数值型数据
lable_dic =dict( zip(class_lable,np.asarray(range(0,len(class_lable)))))
lable_dic

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}

In [65]:
@tf.function
def get_lable(x):
    arr_str = tf.strings.split(x,sep="\\")
    return arr_str[-2] == class_lable

In [66]:
IMG_WIDTH = 224
IMG_HEIGHT = 224
BATCH_SIZE = 5

In [67]:
def decode_img(img):
    img = tf.image.decode_image(img,channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize_with_crop_or_pad(img,IMG_WIDTH,IMG_HEIGHT)
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    return img

In [68]:
def img_map_fun(x):
    lable = get_lable(x)
    img = tf.io.read_file(x)
    img = decode_img(img)
    return img,lable

In [69]:
lable = tf.constant([0,1,2,3,4])
@tf.function
def map_lable_fun(x,y):
    print(y)
    y = lable[y][0]
    return x,y

In [70]:
ds = ds.map(map_func=img_map_fun,num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds = ds.map(map_func=map_lable_fun)

Tensor("y:0", shape=(5,), dtype=bool)


In [71]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000,batch_size = 256):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
    else:
        ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(batch_size)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

In [72]:
ds = prepare_for_training(ds,batch_size = BATCH_SIZE)

In [73]:
image_batch, label_batch = next(iter(ds))

In [79]:
model.compile(loss=tf.losses.sparse_categorical_crossentropy,metrics=["acc"])

In [80]:
model.fit(ds,steps_per_epoch=5,epochs=2)

Train for 5 steps
Epoch 1/2
5/5 [==============================] - 39s 8s/step - loss: 2794.9617 - acc: 0.2000
Epoch 2/2
5/5 [==============================] - 25s 5s/step - loss: 2.8302 - acc: 0.3200
